In [15]:
from datetime import datetime, timezone, timedelta
import pandas as pd
from io import StringIO
import geohash2
import import_ipynb
from evaluation import to_timestamp, request_influxdb, query_tangaras


# Tangara Stations

In [16]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
#start_timestamp = to_timestamp('2023-03-17T00:00:00-05:00')
#start_timestamp = to_timestamp('2023-04-03T00:00:00-05:00')
start_timestamp = to_timestamp('2023-06-12T00:00:00-05:00')

# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
#end_timestamp = to_timestamp('2023-03-31T00:00:00-05:00')
#end_timestamp = to_timestamp('2023-04-04T00:00:00-05:00')
end_timestamp = to_timestamp('2023-06-18T23:59:59-05:00')

print(f'Since: {start_timestamp} Until: {end_timestamp}')


Since: 1686546000000 Until: 1687150799000


In [17]:
# Query Tangaras
query = query_tangaras(start_timestamp, end_timestamp)
print(query)

SELECT DISTINCT(geo) AS "geohash" FROM "fixed_stations_01" WHERE ("geo3" = 'd29') AND time >= 1686546000000ms AND time <= 1687150799000ms GROUP BY "name";


In [18]:
# Get Data Frame Tangaras
def df_tangaras(start_timestamp, end_timestamp):
    # Query Tangaras
    query = query_tangaras(start_timestamp, end_timestamp)
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(query)
    #print(influxdb_request)
    #print(influxdb_request.text)

    # Data Frame Tangaras
    df_tangaras = pd.read_csv(StringIO(influxdb_request.text), sep=",")

    # Remove/Add Columns
    df_tangaras = df_tangaras[['tags', 'geohash']]
    df_tangaras['MAC'] = df_tangaras['tags'].apply(lambda x: x.split('=')[1])
    df_tangaras['GEOLOCATION'] = df_tangaras['geohash'].apply(lambda x: " ".join(str(value) for value in list(geohash2.decode_exactly(x)[0:2])))
    df_tangaras['LATITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[0])
    df_tangaras['LONGITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[1])
    df_tangaras['tags'] = df_tangaras['tags'].apply(lambda x: f"TANGARA_{x[-4:]}")
    df_tangaras.rename(columns={'tags': 'ID', 'geohash': 'GEOHASH'}, inplace=True)
    
    # Date time when query is executed
    #tz = timezone(timedelta(hours=-5))
    #df_tangaras['DATETIME'] = datetime.now(tz=tz)

    # Set Index
    df_tangaras.set_index('ID', inplace=True)
    
    return df_tangaras


In [19]:
# Data Frame Tangaras
tangaras = df_tangaras(start_timestamp, end_timestamp)
tangaras.drop_duplicates(subset=['MAC'], inplace=True)
tangaras.head()

,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
ID,,,,,
TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.3844757080078125,-76.51634216308594
TANGARA_25CE,d29e4r0,D29ESP32DE125CE,3.3789825439453125 -76.54106140136719,3.3789825439453125,-76.54106140136719
TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.4613800048828125,-76.51222229003906
TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.4586334228515625,-76.46415710449219
TANGARA_1282,d29edvs,D29ESP32DED1282,3.4586334228515625 -76.51359558105469,3.4586334228515625,-76.51359558105469


In [20]:
# Save Tangaras into CSV file
tangaras.to_csv('../data/tangaras.csv')